In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from sklearn.datasets import load_digits
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import accuracy_score
from IPython.core.debugger import set_trace
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Data Collection
from sklearn.datasets import fetch_openml

#3 classes:
penguins = fetch_openml(name='penguins')

np.unique(penguins.target)

#load data and targets

X1 = penguins.data
targets = penguins.target
y1 = np.zeros((X1.shape[0], np.unique(penguins.target).size))

np.unique(penguins.target)

array(['Adelie', 'Chinstrap', 'Gentoo'], dtype=object)

In [3]:
# Gradient Optimizer

softmax = lambda z: np.exp(z - z.max(axis=1, keepdims = True)) / np.exp(z - z.max(axis=1, keepdims = True)).sum(axis=1)[:,None]

class Optimizer:
    
    def __init__(self, batch_size=10, learning_rate=0.1, momentum=0, max_iters=1e4, epsilon=1e-8, record_history=False, l1=0, l2=0):
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.momentum = momentum
        self.max_iters = max_iters
        self.epsilon = epsilon
        self.record_history = record_history
        self.l1 = l1
        self.l2 = l2
        if record_history:
            self.w_history = []                 #to store the weight history for visualization
            
    def optimize(self, x, y):
        N,D = x.shape
        w = np.zeros((y.shape[1],D))
        grad = np.inf
        t = 0
        while np.linalg.norm(grad) > self.epsilon and t < self.max_iters:
            grad_prev = grad
            grad = self.gradient(x, y, w) + self.l1 * np.sign(w) + self.l2 * w  # compute the gradient, apply l1 or l2 regularization if specified
            if t > 0:
                grad = self.momentum * grad_prev + (1 - self.momentum) * grad
            if self.learning_rate == 0:           # special case, use decreasing learning rate
                w = w - (1 / (t + 1)) * grad      # weight update step using 1 / (t+1)
            else:
                w = w - self.learning_rate * grad         # weight update step using specified learning rate
            if self.record_history:
                self.w_history.append(w)
            t += 1
        return [w, t, np.linalg.norm(grad)]
    
    def gradient(self, x, y, w):
        index = np.random.choice(x.shape[0], self.batch_size, replace=False) # choose a minibatch based on size
        x_batch = x[index]
        y_batch = y[index]
        z = softmax(x_batch@w.T) - y_batch
        return z.T@x_batch
        

In [4]:
# Cost Function

def cost_fn(x, y, w):                                                   
    z = x@w.T
    J = -((y * z).sum(axis=1) - (z.max(axis=1) + np.log(np.exp(z - z.max(axis=1, keepdims=True)).sum(axis=1)))).mean()
    return J

In [5]:
# Softmax Classifier

class softmax_classifier:
    def __init__(self, verbose=False):
        self.verbose = verbose
    
    def fit(self, x, y, optimizer):
        if x.ndim == 1:
            x = x[:, None]
        N = x.shape[0]
        x = np.column_stack([x,np.ones(N)])
        results = optimizer.optimize(x, y)
        self.w = results[0]
        
        if self.verbose:
            print(f'terminated after {results[1]} iterations, with norm of the gradient equal to {results[2]}')
            print(f'the weight found: {self.w}')
        return self
    
    def predict(self, x):
        if x.ndim == 1:
            x = x[:, None]
        Nt = x.shape[0]
        x = np.column_stack([x,np.ones(Nt)])
        z = softmax(x@self.w.T)
        yh = z.argmax(axis=1)           #predict output
        return yh

In [6]:
# Hyperparameter Optimization
digits=load_digits()
X = digits.data
targets = digits.target
y = np.zeros((X.shape[0], 10))
for i in range(X.shape[0]):
    y[i][targets[i]] = 1
for train, test in KFold().split(X):
    cl = softmax_classifier()
    optimizer = Optimizer(learning_rate = 0, momentum = 0.9, l1=0.1, l2=0.1)
    cl.fit(np.array([X[i] for i in train]), np.array([y[i] for i in train]), optimizer)
    print(accuracy_score([targets[i] for i in test], cl.predict(np.array([X[i] for i in test]))))

0.9027777777777778
0.8444444444444444
0.8997214484679665
0.9108635097493036
0.8272980501392758


In [24]:
# Model for comparison